Functions for analysis

In [52]:
## Functions for reading in data ##

def CheckLeap(Year):  
  # Checking if the given year is leap year  
    if((Year % 400 == 0) or  
     (Year % 100 != 0) and  
     (Year % 4 == 0)):   
        return True  
  # Else it is not a leap year  
    else:  
        return False
    
# Python3 implementation of the approach
days = [31, 28, 31, 30, 31, 30,
        31, 31, 30, 31, 30, 31];
 
# Function to return the day number
# of the year for the given date
def dayOfYear(date):
     
    # Extract the year, month and the
    # day from the date string
    year = (int)(date[0:4]);
    month = (int)(date[5:7]);
    day = (int)(date[8:]);
 
    # If current year is a leap year and the date
    # given is after the 28th of February then
    # it must include the 29th February
    if (month > 2 and year % 4 == 0 and
       (year % 100 != 0 or year % 400 == 0)):
        day += 1;
 
    # Add the days in the previous months
    month -= 1;
    while (month > 0):
        day = day + days[month - 1];
        month -= 1;
    return day;

def dayOfYearAccountingForLeapYear(date):
    
    # If the year is a leap year, this function returns days 1...59 60 61...366
    # If not a leap year, this function returns days 1...59 61...366
     
    # Extract the year, month and the
    # day from the date string
    year = (int)(date[0:4]);
    month = (int)(date[5:7]);
    day = (int)(date[8:]);
 
    # If current year is a leap year and the date
    # given is after the 28th of February then
    # it must include the 29th February
    if (month > 2 and year % 4 == 0 and
       (year % 100 != 0 or year % 400 == 0)):
        day += 1;
 
    # Add the days in the previous months
    month -= 1;
    while (month > 0):
        day = day + days[month - 1];
        month -= 1;
        
    if not ((year % 4 == 0 and
       (year % 100 != 0 or year % 400 == 0))):
        if day >=60:
            day +=1
    
    return day;



def findYearIndices(startYear,endYear,dates,yearsToSkip=[]):
    # Start and end year correspond to the year in the spring of the winter
    # The first data point is in Jan 1930, so the start year is 1930
    # ex. a start of year of 2001 would start with the winter season of 2000-2001
    # inclusive of endYear
    
    yearRange = np.arange(startYear,endYear+1)
    
    for ySkip in yearsToSkip:
        yearRange = np.delete(yearRange,np.where(yearRange==ySkip))
    
    yearIndices = []
    for y in yearRange:
        currYearIndices = []
        for di in range(len(dates)):
            cond1 = float(dates[di][0:4]) == y and int(dates[di][5:7])<9
            cond2 = float(dates[di][0:4]) == y-1 and int(dates[di][5:7])>=9
            if cond1 or cond2:
                currYearIndices = currYearIndices + [di]
        yearIndices = yearIndices + [currYearIndices]
    return yearIndices,yearRange

def fillInClim(datesByWinter,snowDepthByYears):
    '''Fill in each missing day for each year in the climatology with nan'''
    
    for iy in range(len(datesByWinter)):
        
        for dayVal in range(1,367):
            checkIfIn = sum(np.isin(datesByWinter[iy],dayVal))
            if checkIfIn == 0:
                # Find the index of the previous value
                
                if dayVal==1:
                    indForMissingDay = 0
                else:
                    indForMissingDay = tuple(np.argwhere(datesByWinter[iy] == dayVal-1)[0])
                datesByWinter[iy] = np.insert(datesByWinter[iy],indForMissingDay,dayVal)
                snowDepthByYears[iy] = np.insert(snowDepthByYears[iy],indForMissingDay,np.nan)
    return snowDepthByYears


def fillInNans(yearlyTimeSeries):
    '''fft can only be used when no NANs are present in the data. Fill in missing values using linear interpolation.
    Note that this function sets snowpack to 0 before October 15 and after May 15. While this does 0-out some data,
    I don't think the impacts are significant.
    '''
    x  = copy.deepcopy(yearlyTimeSeries)
    x[dayOfYear('2001-05-15')+122-1:] = 0
    x[:dayOfYear('2000-10-01')+122-366-1] = 0
    xi = np.arange(len(x))

    mask = np.isfinite(x)
    xfiltered = np.interp(xi, xi[mask], x[mask])

    return xfiltered

def ampFromFFT(yearlyTimeSeries):
    '''This function returns the amplitude of the Fourier components as a function of frequency.
    It assumes daily sampling.
    '''
    sampleFrequency = 366 # samples per year
    timeOverYear = np.arange(1,367)
    freq = np.arange(0,sampleFrequency,sampleFrequency/len(timeOverYear))
    N = len(yearlyTimeSeries) # Number of samples
    
    # Amplitude Spectrum from FFT
    xk = abs(np.fft.fft(yearlyTimeSeries))/N; # Two-sided amplitude
    xk = xk[0:int(N/2)]; # One-sided
    xk[1:-2] = 2*xk[1:-2]; # Double values except for DC and Nyquist (no longer two-sided)

    freqHalf = freq[1:int(N/2+1)]
    
    return xk,freqHalf

def THD(amplitudes):
    '''Take in a list of Fourier component amplitudes and calculate the total harmonic distortion.
    The data is assumed to have the DC component as the first index.
    '''
    withNoDC = amplitudes[1:]
    V1 = withNoDC[0]
    num = np.sqrt(np.sum(withNoDC[1:]**2))
    
    return num/V1


# Moving Average
def movingaverage(y, N):
    # N is the window length
    y_padded = np.pad(y, (N//2, N-1-N//2), mode='edge')
    y_smooth = np.convolve(y_padded, np.ones((N,))/N, mode='valid') 
    return y_smooth

# Calculate median and interquartile range
def medIQR(a):
    return np.median(a), np.percentile(np.array(maxSnowPackByWinter),[25, 75])

# Calculate mean and standard deviation
def meanSTD(a):
    return np.mean(a), np.std(a)

In [11]:
# Calculate the number of dips in the snowpack over the course of the winter
# a dip is defined as any time the first derivative drops below 0

def numOfDips(yearTimeSeries,thresh=-1):
    '''Counts the number of times the snowpack decreases throughout the winter'''
    
    currYearGrad = np.gradient(yearTimeSeries)

    dipCount = 0
    for j in range(len(currYearGrad)-1):
        if currYearGrad[j]>=0 and currYearGrad[j+1]<0:
            if currYearGrad[j+1]<=thresh:
                dipCount += 1
    return dipCount

In [25]:
def inputToDatasetName(strList):
    '''This function takes in a list of names and returns a list of names for stations in the dataset'''
    
    nameDict = {'Lancaster':'LANCASTER, NH US',
                'Berlin':'BERLIN, NH US',
                'Errol':'ERROL, NH US',
                'Noth Stratford':'NORTH STRATFORD, NH US',
                'Hanover':'HANOVER, NH US',
                'First Connecticut Lake':'FIRST CONNECTICUT LAKE, NH US',
                'Bethlehem':'BETHLEHEM, NH US',
                'Benton':'BENTON 5 SW, NH US',
                'Old North Conway':'CONWAY 1 N, NH US',
                'York Pond':'YORK POND, NH US',
                'Jefferson':'JEFFERSON, NH US',
                'North Conway':'NORTH CONWAY, NH US',
                'Pinkham Notch':'PINKHAM NOTCH, NH US',
                'Mount Washington':'MOUNT WASHINGTON, NH US'
               }
    
    newList = [nameDict[inpt] for inpt in strList]
    
    return newList

In [10]:
def processFullDataset(stationData,variable,stationName):
    '''This function takes in the data for a station and returns a cleaned up version with a time series of the
    chosen variable throughout each year. Variables of interest include:
    - date: DATE
    - snow depth: SNWD 
    - snow fall: SNOW
    - daily max temp: TMAX
    - daily min temp: TMIN '''
    
    ###################################################
    # Get dates and variable of choice
    
    dates = stationData.DATE.to_numpy()
    variableOfChoice = stationData[variable].to_numpy()
    
    ###################################################
    
    # Find the indices in the data corresponding to each year
    # we want to track winter seasons, thus the year will be defined from Sept to Sept

    lastYearInClim = int(dates[-1][:4])
    startingYear = int(dates[0][:4])
    skipYears = []
    
    # because of data availability in certain stations, we can modify these
    if stationName == 'York Pond':
        startingYear = 1971 # spotty coverage before this
    elif stationName == 'Pinkham Notch':
        startingYear = 1931 # 1930 is not complete
    elif stationName == 'North Conway':
        startingYear = 1960 # 1959 is not complete
        skipYears = [1974] # bad data in this year
    
    indicesByYear,yearRange = findYearIndices(startingYear,lastYearInClim,dates,skipYears)
    
    ###################################################
    
    # Select the variable of choice in each year

    variableOfChoiceByYears = []
    datesByWinter = []

    for iyear in indicesByYear:
        variableOfChoiceInYear = variableOfChoice[iyear]
        
        # Convert to SI units:
        if variable == 'SNWD' or variable == 'SNOW':
            variableOfChoiceInYear = variableOfChoiceInYear * 2.54 # in to cm
        elif variable == 'TMAX' or variable == 'TMIN':
            variableOfChoiceInYear = (variableOfChoiceInYear-32) * (5/9) # F to C
        
        variableOfChoiceByYears = variableOfChoiceByYears + [variableOfChoiceInYear]

        datesByWinter = datesByWinter + [dates[iyear]]

    # Turn calendar date into day of year (with Sept 1 = day 1)
    for iy in range(len(datesByWinter)):
        for iday in range(len(datesByWinter[iy])):

            datesByWinter[iy][iday] = dayOfYearAccountingForLeapYear(datesByWinter[iy][iday])+122

            if datesByWinter[iy][iday]>366 and datesByWinter[iy][iday] != datesByWinter[iy][-1]:
                datesByWinter[iy][iday] = datesByWinter[iy][iday]-366
             
    ###################################################        
            
    # Calculate the average over the years as well

    dayAverageVariableOfChoice = []
    for day in range(1,367):
        runningListForMean = []
        for iy in range(len(datesByWinter)):
            if day in datesByWinter[iy]:
                # Find the index of that day in datesByWinter[iy]
                indexOfParticularDay = np.nonzero(datesByWinter[iy] == day)[0][0]
                runningListForMean = runningListForMean + [variableOfChoiceByYears[iy][indexOfParticularDay]]
        dayAverageVariableOfChoice = dayAverageVariableOfChoice + [np.nanmean(np.array(runningListForMean))]
        
    ###################################################
    
    dictToReturn = {}
    dictToReturn['avg'] = {'dates':np.arange(1,367),'data':np.array(dayAverageVariableOfChoice)}
    
    for ind, year in enumerate(yearRange):
        
        dictToReturn[year] = {'dates':datesByWinter[ind],'data':variableOfChoiceByYears[ind]}
        
    return dictToReturn

In [46]:
def calcSnowToDate(snowfallDict):
    
    # Copy the input
    snowfallDict1 = copy.deepcopy(snowfallDict)
    
    snowToDateDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationSnowfall = snowfallDict1[currName]
        
        for yearKey in stationSnowfall.keys():
            yearData = stationSnowfall[yearKey]
            stationSnowfall[yearKey]['data'] = np.nancumsum(yearData['data'])
        
        snowToDateDict[currName] = stationSnowfall
        
    return snowToDateDict

In [58]:
def calcSeasonTotalSnowfall(snowToDateDict):
    
    # Copy the input
    snowToDateDict1 = copy.deepcopy(snowToDateDict)
    
    seasonTotalSnowfallDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationData = snowToDateDict1[currName]
        
        for yearKey in stationData.keys():
            yearData = stationData[yearKey]
            
            if len(yearData['data'])==0:
                stationData[yearKey] = np.nan
            else:
                stationData[yearKey] = np.nanmax(yearData['data'])
        
        seasonTotalSnowfallDict[currName] = stationData
    
    return seasonTotalSnowfallDict

In [59]:
def calcMaxSnowDepth(snowDepthDict):
    
    # Copy the input
    inputDict1 = copy.deepcopy(snowDepthDict)
    
    returnDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationData = inputDict1[currName]
        
        for yearKey in stationData.keys():
            yearData = copy.deepcopy(stationData[yearKey])
            
            if len(yearData['data'])==0:
                stationData[yearKey]['data'] = np.nan
                stationData[yearKey]['dates'] = np.nan
            else:
                stationData[yearKey]['data'] = np.nanmax(yearData['data'])
                stationData[yearKey]['dates'] = yearData['dates'][np.nanargmax(yearData['data'])]

        returnDict[currName] = stationData    
        
    return returnDict   

In [71]:
def calcTHD_forDict(snowDepthDict):
    '''Calculate the Total Harmonic Distortion of the Snowdepth or mean temperature over the course of
    each winter. Note this calculation requires filling in the NANs in the data.
    This is done within this function.
    '''
    
    # Copy the input
    inputDict1 = copy.deepcopy(snowDepthDict)
    
    returnDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationData = inputDict1[currName]
        
        for yearKey in stationData.keys():
            yearData = copy.deepcopy(stationData[yearKey])
            
            snowDepthDataForThisYear = yearData['data']
            
            if len(snowDepthDataForThisYear)==0:
                stationData[yearKey] = np.nan
            
            else:
                # Fill in NANs
                filledNANs = fillInNans(snowDepthDataForThisYear)

                # Calculate the amplitude of the Fourier Components for each year
                amps = ampFromFFT(filledNANs)[0]

                # Calculate the total harmonic distortion 
                thdForYear = THD(amps)

                stationData[yearKey] = thdForYear
        
        returnDict[currName] = stationData
    
    return returnDict

In [13]:
def calcDaysAboveThresh(snowDepthDict,threshold):
    
    # Copy the input
    inputDict1 = copy.deepcopy(snowDepthDict)
    
    returnDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationData = inputDict1[currName]
        
        for yearKey in stationData.keys():
            yearData = copy.deepcopy(stationData[yearKey])
            
            snowDepthDataForThisYear = yearData['data']
            
            numDaysAboveThreshold = len(snowDepthDataForThisYear[snowDepthDataForThisYear>=threshold])
            
            stationData[yearKey] = numDaysAboveThreshold
        
        returnDict[currName] = stationData
    
    return returnDict

In [79]:
def calcDailyMeanTemp(tMinDict,tMaxDict):
    
    # Copy the inputs
    minT = copy.deepcopy(tMinDict)
    maxT = copy.deepcopy(tMaxDict)
    
    returnDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationDataMin = minT[currName]
        stationDataMax = maxT[currName]
        
        for yearKey in stationDataMin.keys():
            yearDataMin = copy.deepcopy(stationDataMin[yearKey])
            yearDataMax = copy.deepcopy(stationDataMax[yearKey])
            
            actualDataMin = yearDataMin['data']
            actualDataMax = yearDataMax['data']
            
            mean = (actualDataMin + actualDataMax)/2
            
            stationDataMax[yearKey]['data'] = mean # the 'max' variable now has mean values
            
        returnDict[currName] = stationDataMax
        
    return returnDict

In [9]:
def calcMeanDJFMTemp(meanTemperatureDict):
    
    # Copy the input
    inputDict1 = copy.deepcopy(meanTemperatureDict)
    
    returnDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationData = inputDict1[currName]
        
        for yearKey in stationData.keys():
            yearData = copy.deepcopy(stationData[yearKey])
            
            dec1Ind = dayOfYear('2000-12-01')+122-366-1
            mar31Ind = dayOfYear('2001-03-31')+122-1
            
            yearDataDJFM = yearData['data'][dec1Ind:mar31Ind+1]
            
            djfmMean = np.nanmean(yearDataDJFM)
            
            #if djfmMean>50: # this most likely indicates bad data
                #djfmMean = np.nan
            
            stationData[yearKey] = djfmMean
        
        returnDict[currName] = stationData

    return returnDict

In [75]:
def calcPDD(timeSeries):
    '''Note that timeSeries must be in C'''
    return np.nansum(timeSeries[timeSeries>=0])


def calcPDDinDJFM(meanTemperatureDict):
    
    # Copy the input
    inputDict1 = copy.deepcopy(meanTemperatureDict)
    
    returnDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationData = inputDict1[currName]
        
        for yearKey in stationData.keys():
            yearData = copy.deepcopy(stationData[yearKey])
            
            dec1Ind = dayOfYear('2000-12-01')+122-366-1
            mar31Ind = dayOfYear('2001-03-31')+122-1
            
            yearDataDJFM = yearData['data'][dec1Ind:mar31Ind+1]
            
            pdd = calcPDD(yearDataDJFM)
            
            stationData[yearKey] = pdd
        
        returnDict[currName] = stationData

    return returnDict

In [5]:
def calcSSM(snowDepthDict):
    '''From Hatchett 2021, SSM is a metric from -1 to 1 describing how ephemeral a snow pack
    is. -1 indicates fully ephemeral, and 1 indicates fully seasonal.'''
    
    # Copy the input
    inputDict1 = copy.deepcopy(snowDepthDict)
    
    returnDict = {}
    
    for iname,currName in enumerate(stationNames):
        stationData = inputDict1[currName]
        
        for yearKey in stationData.keys():
            yearData = copy.deepcopy(stationData[yearKey])
            
            snowDepthDataForThisYear = yearData['data']
            
            stationData[yearKey] = SSMofTimeSeries(snowDepthDataForThisYear)
        
        returnDict[currName] = stationData
    
    return returnDict


def SSMofTimeSeries(timeSeries):
    
    # Find where snowpack is greater than zero
    condition = timeSeries>0
    
    # Find the number of consecutive days with this condition
    consecDayGroupings = np.diff(np.where(np.concatenate(([condition[0]],
                                     condition[:-1] != condition[1:],
                                     [True])))[0])[::2]
    
    totalNumberOfDaysWithSnow = np.sum(consecDayGroupings)
    
    daysSeasonalSnow = np.sum(consecDayGroupings[consecDayGroupings>=60])
    
    daysEphemeralSnow = np.sum(consecDayGroupings[consecDayGroupings<60])
    
    return (daysSeasonalSnow-daysEphemeralSnow)/totalNumberOfDaysWithSnow




In [14]:
def axisLabels(variable):
    '''Function returns string to be used as an axis label in a plot'''

    if variable == snowToDateDict:
        return 'Snowfall to Date (cm)'
    elif variable == seasonTotalSnowfallDict:
        return 'Season Total Snowfall (cm)'
    elif variable == maxSnowDepthDict:
        return 'Peak Snow Depth (cm)'
    elif variable == thdDict:
        return 'Total Harmonic Distortion (snow depth)'
    elif variable == numDaysAboveThresh:
        return 'Number of days above {} cm'.format(thresholdSnwDepth)
    elif variable == ssmDict:
        return 'Snow Seasonality Metric'
    elif variable == meanTemperatureDict:
        return 'Daily Mean Temperature (C)'
    elif variable == meanTempTHDDict:
        return 'Total Harmonic Distortion (Temperature)'
    elif variable == meanTempDJFMDict:
        return 'DJFM mean temperature (C)'
    elif variable == ppdDict:
        return r'PDD (C$\cdot$days)'
